# Praktikum 4 
## Klasifikasi dengan AAN

### Data Pra-processing
**Langkah 1 - Import Libary**

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

**Langkah 2 - Load Data**

In [26]:
dataset = pd.read_csv('./Dataset/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [27]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


**Langkah 3 - Encoding Data Kategorikal**

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [29]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


**Langkah 4 - Encoding Kolom "Geography" dengan One Hot Encoder**

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [31]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


**Langkah 5 - Split Data**

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Langkah 6 - Scaling Fitur**

In [33]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Membuat Model ANN

**Langkah 1 - Inisiasi Model ANN**

In [34]:
ann = tf.keras.models.Sequential()

**Langkah 2 - Membuat Input Layer dan Hidden Layer Pertama**

In [35]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

**Langkah 3 - Membuat Hidden Layer Kedua**

In [36]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

**Langkah 4 - Membuat Output Layer**

In [37]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training Model

**Langkah 1 - Compile Model (Menyatukan Arsitektur) ANN**

In [38]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

**Langkah 2 - Fitting Model**

In [39]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
161/250 [==================>...........] - ETA: 0s - loss: 0.6427 - accuracy: 0.6990

250/250 [==============================] - 1s 2ms/step - loss: 0.6031 - accuracy: 0.7271
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4878 - accuracy: 0.7854
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4491 - accuracy: 0.8005
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4334 - accuracy: 0.8109
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4257 - accuracy: 0.8196
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8253
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4158 - accuracy: 0.8274
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4120 - accuracy: 0.8299
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4083 - accuracy: 0.8303
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4039 - accuracy: 0.8322
E

### Memodelkan Data Baru dan Buat Prediksi

In [40]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 124ms/step
[[False]]


### Prediksi Dengan Data Testing

In [41]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Cek Akurasi dan Confusion Matrix

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1496   99]
 [ 188  217]]


0.8565